# Spin Configuration Generator

In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os

### Loading Datasets

You can load different datasets by changing the dataset path for testing, trainging, and benchmarking

In [26]:
L = 32

In [27]:
# Set these variables to the appropriate directory paths

test_dir = '../data/{}-{}/binary_class/test/'.format(L,L)
train_dir = '../data/{}-{}/binary_class/train/'.format(L,L)

# Set these variables to the appropriate directory paths
benchmark_dir = '../data/{}-{}/temp_class/test/'.format(L,L)

In [28]:
# data transforms
dset_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()])

#create datasets using ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=dset_transform)
test_data = datasets.ImageFolder(test_dir, transform=dset_transform)
benchmark_data = datasets.ImageFolder(benchmark_dir, transform=dset_transform)

batch_size = 64
# create dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

### Helper functions

Thanks to Teal Witter for code for these helper functions https://github.com/rtealwitter/dl-demos/blob/b537a5dd94953ea656a2140227af78f67c042540/demo11-conditional-gan.ipynb

In [29]:
def to_onehot(x, num_classes=2):
    assert isinstance(x, int) or isinstance(x, (torch.LongTensor, torch.cuda.LongTensor))
    if isinstance(x, int):
        c = torch.zeros(1, num_classes).long()
        c[0][x] = 1
    else:
        x = x.cpu()
        c = torch.LongTensor(x.size(0), num_classes)
        c.zero_()
        c.scatter_(1, x, 1) # dim, index, src value
    return c


In [30]:
def get_sample_image(G, DEVICE, n_noise=100):
    img = np.zeros([L*2, L*2])
    for j in range(2):
        c = torch.zeros([2, 2]).to(DEVICE)
        c[:, j] = 1
        z = torch.randn(2, n_noise).to(DEVICE)
        y_hat = G(z,c).view(2, L, L)
        result = y_hat.cpu().data.numpy()
        img[j*L:(j+1)*L] = np.concatenate([x for x in result], axis=-1)
    return img

In [31]:
input_size = [L,L]
hidden_layers = 4
layer_size = 128
num_classes = 2

## Architecture

Define the architecture of the generator and discriminator

In [32]:
def linear_block(layer_size, num_layers):
    layers = []
    for i in range(num_layers):
        layers.append(nn.Linear(layer_size, layer_size))
        layers.append(nn.LeakyReLU())
    return layers

In [33]:
class Generator(nn.Module):
    def __init__(self, image_dim, hidden_layers, layer_size, num_classes, input_size=100):
        super(Generator, self).__init__()
        self.image_size = np.prod(image_dim)
        self.input_layer = [nn.Linear(input_size + num_classes, layer_size)]
        self.hidden_layers = linear_block(layer_size, hidden_layers)
        self.output_layer = [nn.Linear(layer_size, self.image_size), nn.Tanh()]
        self.model = nn.Sequential(*self.input_layer, *self.hidden_layers, *self.output_layer)

    def forward(self, x, c):
        x, c = x.view(x.size(0), -1), c.view(c.size(0), -1).float()
        v = torch.cat([x, c], dim=1)
        return self.model(v)


In [34]:
class Discriminator(nn.Module):
    def __init__(self, image_dim, hidden_layers, layer_size, num_classes):
        super(Discriminator, self).__init__()
        self.image_size = np.prod(image_dim)
        self.input_layer = [nn.Linear(self.image_size + num_classes, layer_size)]
        self.hidden_layers = linear_block(layer_size, hidden_layers)
        self.output_layer = [nn.Linear(layer_size, 1), nn.Sigmoid()]
        self.model = nn.Sequential(*self.input_layer, *self.hidden_layers, *self.output_layer)

    def forward(self, x, c):
        x, c = x.view(x.size(0), -1), c.view(c.size(0), -1).float()
        v = torch.cat([x, c], dim=1)
        return self.model(v)

## Setup and Training

In [35]:
MODEL_PATH = './models/{}-{}-GAN/'.format(L,L)
SAMPLE_PATH = '../generated_samples/{}-{}-GAN/'.format(L,L)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

D = Discriminator(input_size, hidden_layers, layer_size, num_classes).to(DEVICE)
G = Generator(input_size, hidden_layers, layer_size, num_classes).to(DEVICE)

max_epoch = 50
step = 0
n_noise = 100 # noise dimension

criterion = nn.BCELoss()
D_optimizer = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_optimizer = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))


# We will denote real images as 1s and fake images as 0s
# This is why we needed to drop the last batch of the data loader
D_labels = torch.ones([batch_size, 1]).to(DEVICE) # Discriminator label: real
D_fakes = torch.zeros([batch_size, 1]).to(DEVICE) # Discriminator Label: fake

In [36]:
if not os.path.exists(SAMPLE_PATH):
    os.makedirs(SAMPLE_PATH)

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

for epoch in range(max_epoch):
    for i, (images, labels) in enumerate(train_loader):
        # Train Discriminator
        x = images.to(DEVICE)
        y = labels.view(batch_size, 1) # add singleton dimension so batch_size x 1
        y = to_onehot(y, num_classes=2).to(DEVICE)

        x_outputs = D(x, y)
        D_x_loss = criterion(x_outputs, D_labels)

        z = torch.randn(batch_size, n_noise).to(DEVICE)
        z_outputs = D(G(z, y), y)
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss

        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train Generator
        z = torch.randn(batch_size, n_noise).to(DEVICE)
        z_outputs = D(G(z, y), y)
        G_loss = -1 * criterion(z_outputs, D_fakes)

        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        if step % 100 == 0:
            print('Epoch: {}, Step: {}, D_loss: {}, G_loss: {}'.format(epoch, step, D_loss.item(), G_loss.item()))
            
        if step % 500 == 0:
            G.eval()
            img = get_sample_image(G, DEVICE, n_noise)
            plt.imsave(SAMPLE_PATH + 'sample-{}-{}.png'.format(epoch, step), img, cmap='gray')
            G.train()

        step += 1


Epoch: 0, Step: 0, D_loss: 1.3846461772918701, G_loss: -0.6676919460296631
Epoch: 1, Step: 100, D_loss: 2.182219982147217, G_loss: -1.258021593093872
Epoch: 3, Step: 200, D_loss: 1.327312707901001, G_loss: -0.327745258808136
Epoch: 4, Step: 300, D_loss: 1.253021478652954, G_loss: -0.46114084124565125
Epoch: 6, Step: 400, D_loss: 1.2940622568130493, G_loss: -0.444781631231308
Epoch: 7, Step: 500, D_loss: 1.4484095573425293, G_loss: -0.634724497795105
Epoch: 9, Step: 600, D_loss: 1.3560107946395874, G_loss: -0.5406897068023682
Epoch: 10, Step: 700, D_loss: 1.2805793285369873, G_loss: -0.5863516330718994
Epoch: 12, Step: 800, D_loss: 1.2971081733703613, G_loss: -0.6525602340698242
Epoch: 14, Step: 900, D_loss: 1.1712019443511963, G_loss: -0.4854898154735565
Epoch: 15, Step: 1000, D_loss: 1.2363122701644897, G_loss: -0.7394949793815613
Epoch: 17, Step: 1100, D_loss: 1.1473584175109863, G_loss: -0.4680593013763428
Epoch: 18, Step: 1200, D_loss: 1.277485728263855, G_loss: -0.5198801755905151